In [1]:
import pandas as pd
import os
from datetime import datetime


### Set data sources and parameters

In [2]:
apc_dir = r'C:\data\OneDrive\WSP O365\chattanooga-modeling - RTP TDM Update\Data\CARTA_PassengerData\CARTA APC DATA 2019'

gtfs_dir = r'C:\data\OneDrive\WSP O365\chattanooga-modeling - RTP TDM Update\Data\TransitUpdate2\GTFS'
# gtfs_dir = r'C:\apps\client_chattanooga_rtp_update\Data\GTFS\carta_2019_08_15'

datatypes = {'STOP_ID': str, 'ROUTE_NUMBER':str, 'stop_id': str}

In [3]:
## APC to GTFS
route_xref = pd.DataFrame(data=[
    ['1.40',   'Route #1 am HC:1',  '1',   'ALTON PARK'],
    ['10.10',  'Route #10A',        '10A', 'AVON'],
    ['10.60',  'Route #10C',        '10C', 'CAMPBELL'],
    ['10.90',  'Route #10G 58:10G', '10G', 'GLENWOOD'],
    ['13.00',  'Route #13',         '13',  'ROSSVILLE'],
    ['14.00',  'Route #14',         '14',  'MOCS EXPRESS'],
    ['15.00',  'Route #15',         '15',  'ST. ELMO'],
    ['16.40',  'Route #16 In:16',   '16',  'NORTHGATE OB'],
    ['19.00',  'Route #19',         '19',  'CROMWELL ROAD'],
    ['2.00',   'Route #2',          '2',   'NORTH CHATT'],
    ['21.00',  'Route #21',         '21',  'GOLDEN GATEWAY'],
    ['28.00',  'Route #28',         '28',  'AMNICOLA HWY CHATT STATE'],
    ['33.00',  'Route #DTS:33',     '33',  'DOWNTOWN SHUTTLE'],
    ['34.00',  'Route #34',         '34',  'NORTH SHORE SHUTTLE'],
    ['4.00',   'Route #4',          '4',   'EASTGATE/HAMILTON PL'],
    ['7.00',   'Route #7',          '7',   'CHATTANOOGA HOUSING AUTHORITY'],
    ['780.00', '3:3',               '3',   'ENTERPRISE SOUTH'],
    ['8.00',   'Route #8',          '8',   'EASTDALE'],
    ['9.00',   'Route #9',          '9',   'EAST LAKE'],
    ],
    columns=['ROUTE_NUMBER', 'ROUTE_NAME', 'route_id', 'route_short_name']
)

In [4]:
### AUGUST
apc_file = 'AUG2019_RIDECHECK_DATA.TXT'

## Tue, Wed, Thur
dates_pick = [
             1,
    6,   7,  8,
    13, 14, 15,
    20, 21, 22,
    27, 28, 29
]

dates_pick = ['8/{}/2019 0:00:00'.format(d) for d in dates_pick]

### Load APC and GTFS Data

In [80]:
apc = pd.read_csv(os.path.join(apc_dir, apc_file), dtype=datatypes)
apc = pd.merge(apc, route_xref[['ROUTE_NUMBER', 'route_id', 'route_short_name']], on='ROUTE_NUMBER', how='left')
# list(apc)
# apc = apc[['ROUTE_NUMBER', 'ROUTE_NAME','STOP_ID', 'MAIN_CROSS_STREET']].copy()

apc_stops = apc.groupby(['STOP_ID', 'MAIN_CROSS_STREET'])['STOP_ID'].count().reset_index(name='count')
apc_stops['MAIN_CROSS_STREET'] = apc_stops['MAIN_CROSS_STREET'].str.replace('/', ' + ')

gtfs = pd.read_csv(os.path.join(gtfs_dir, 'stops.txt'), dtype=datatypes)
gtfs = gtfs[['stop_id', 'stop_name']]

C:\Anaconda3\envs\chattanooga\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [81]:
# apc[['ROUTE_NUMBER', 'ROUTE_NAME']].groupby(['ROUTE_NUMBER', 'ROUTE_NAME'])['ROUTE_NUMBER'].count().reset_index(name='_count')

##### Verify APC to GTFS Stop_ID using intersection data
Note that non-matching records are still being counted with the 'ROUTE_NAME' attribute

In [92]:
stops_check = pd.merge(apc_stops, gtfs, left_on='STOP_ID', right_on='stop_id', how='left')

matches = stops_check[stops_check['stop_id'].notna()]
not_matches = stops_check[stops_check['stop_id'].isna()]

print('APC Stations: {:,}'.format(len(apc_stops)))
print('GTFS Stations: {:,}'.format(len(gtfs)))
print('APC - GTFS Stations difference: {:,}'.format(len(apc_stops) - len(gtfs)))

print('Station EXACT Matches: {:,} ratio: {:.2f}'.format(len(matches), len(matches) / len(apc_stops)))

not_matches['MAIN_CROSS_STREET'] = not_matches['MAIN_CROSS_STREET'].str.replace(' ', '')
not_matches['stop_name'] = not_matches['stop_name'].str.replace(' ', '')
close_matches = not_matches[not_matches['MAIN_CROSS_STREET']==not_matches['stop_name']]
print('Station CLOSE Matches: {:,}'.format(len(close_matches)))

print('Station NOT Matches: {:,}'.format(len(not_matches)-len(close_matches)))

null_stop_id = not_matches[not_matches['stop_name'].isna()]
print('APC Stations ID NOT in GTFS: {:,}'.format(len(null_stop_id)))

# print(not_matches['MAIN_CROSS_STREET'].unique())

APC Stations: 1,240
GTFS Stations: 1,181
APC - GTFS Stations difference: 59
Station EXACT Matches: 1,153 ratio: 0.93
Station CLOSE Matches: 0
Station NOT Matches: 87
APC Stations ID NOT in GTFS: 87


C:\Anaconda3\envs\chattanooga\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Anaconda3\envs\chattanooga\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [8]:
# matches.sample(10)

### Route level average data boardings by TOD
Weekdays:
    Tue
    Wed
    Thurs
    
August 2019 (13 obs)

Populate table 'Transit Route Boardings':  
https://wsponline.sharepoint.com/:w:/r/sites/US-chattanoogamodeling/RTP%20TDM%20Update/Deliverables/Task%203_Model%20Validation/Chattanooga%20TPO%20Model%20Validation%20Report.docx

##### Daily average

In [36]:
apc_pick = apc[apc['SURVEY_DATE'].isin(dates_pick)]
apc_pick = apc_pick[['SERIAL_NUMBER', 'SURVEY_DATE', 'ROUTE_NUMBER', 'route_id', 'route_short_name']]

weekdays = apc_pick.groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name', 'SURVEY_DATE']).count()\
                   .groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name']).mean()

weekdays = round(weekdays).astype(int)
weekdays = weekdays.reset_index()
weekdays.columns=['ROUTE_NUMBER', 'route_id', 'route_short_name', '_count']
weekdays['ROUTE_NUMBER_num'] = weekdays['ROUTE_NUMBER'].astype(float)
weekdays = weekdays.sort_values(by='ROUTE_NUMBER_num')#.to_clipboard()

weekdays.drop(columns='ROUTE_NUMBER_num')

,ROUTE_NUMBER,route_id,route_short_name,_count
0,1.40,1,ALTON PARK,2565
8,2.00,2,NORTH CHATT,342
13,4.00,4,EASTGATE/HAMILTON PL,7200
14,7.00,7,CHATTANOOGA HOUSING AUTHORITY,344
16,8.00,8,EASTDALE,1552
17,9.00,9,EAST LAKE,2763
1,10.10,10A,AVON,1419
2,10.60,10C,CAMPBELL,553
3,10.90,10G,GLENWOOD,1924
4,13.00,13,ROSSVILLE,1695


##### Update Time Of Day periods

In [37]:
## CHECKS on TOD
# apc[['SURVEY_DATE', 'TRIP_START_TIME', 'TIME_PERIOD', 'TIME_SCHEDULED', 'TIME_ACTUAL_ARRIVE', 'TIME_ACTUAL_DEPART',]].sample(5)
# apc['TIME_PERIOD'].unique()

# test[(test['TIME_PERIOD']=='AM Early')&(test['ROUTE_NUMBER']==7)].groupby('SURVEY_DATE').count()
# test[(test['TIME_PERIOD']=='PM Late')&(test['ROUTE_NUMBER']==2)].groupby('SURVEY_DATE').count()

# apc.groupby(['TIME_PERIOD']).agg({'TRIP_START_TIME':['min', 'max']})

In [38]:
tod_windows = {
    'AM':{'from': '6', 'to': '9'},
    'PM':{'from': '15', 'to': '18'}
}

apc['trip_start_astime'] = [datetime.strptime(t, '%m/%d/%Y %H:%M:%S').time() for t in apc['TRIP_START_TIME']] 

apc['TOD'] = 'OP'  ## All other times are set to Off-Peak

for t in tod_windows.keys():
    apc.loc[apc['trip_start_astime'].between(
        datetime.strptime((tod_windows[t]['from']), '%H').time(),
        datetime.strptime((tod_windows[t]['to']), '%H').time()
        ), 'TOD'] = t

##### Time of Day average

In [39]:
apc_pick = apc[apc['SURVEY_DATE'].isin(dates_pick)]
apc_pick = apc_pick[['SERIAL_NUMBER', 'SURVEY_DATE', 'ROUTE_NUMBER', 'route_id', 'route_short_name', 'TOD']]

weekdays = apc_pick.groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name', 'SURVEY_DATE', 'TOD']).count()\
                   .groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name', 'TOD']).mean().unstack()

weekdays = round(weekdays).astype(int)
weekdays = weekdays.reset_index()
weekdays.columns=['ROUTE_NUMBER', 'route_id', 'route_short_name', 'AM', 'OP', 'PM']
weekdays['ROUTE_NUMBER_num'] = weekdays['ROUTE_NUMBER'].astype(float)
weekdays = weekdays.sort_values(by='ROUTE_NUMBER_num')#.to_clipboard()

weekdays.drop(columns='ROUTE_NUMBER_num')

,ROUTE_NUMBER,route_id,route_short_name,AM,OP,PM
0,1.40,1,ALTON PARK,518,1568,479
8,2.00,2,NORTH CHATT,73,170,140
13,4.00,4,EASTGATE/HAMILTON PL,1614,3998,1588
14,7.00,7,CHATTANOOGA HOUSING AUTHORITY,44,264,55
16,8.00,8,EASTDALE,337,876,395
17,9.00,9,EAST LAKE,572,1618,574
1,10.10,10A,AVON,318,818,283
2,10.60,10C,CAMPBELL,207,255,169
3,10.90,10G,GLENWOOD,424,1065,436
4,13.00,13,ROSSVILLE,411,832,596


### Major Station Boardings

##### For each Day

In [59]:
apc_pick = apc[apc['SURVEY_DATE'].isin(dates_pick)]
apc_pick[['STOP_ID', 'MAIN_CROSS_STREET', 'SURVEY_DATE']]
station_boards = apc_pick.groupby(['STOP_ID', 'MAIN_CROSS_STREET', 'SURVEY_DATE'])['SERIAL_NUMBER']\
                         .count().reset_index(name='BOARDS')
station_boards.sort_values(by='BOARDS', ascending=False).head(20)

,STOP_ID,MAIN_CROSS_STREET,SURVEY_DATE,BOARDS
236,100075,SPN,8/14/2019 0:00:00,235
237,100075,SPN,8/15/2019 0:00:00,225
239,100075,SPN,8/7/2019 0:00:00,225
234,100075,SPN,8/1/2019 0:00:00,221
6894,742,MARKET/TVA,8/7/2019 0:00:00,218
240,100075,SPN,8/8/2019 0:00:00,217
238,100075,SPN,8/6/2019 0:00:00,214
235,100075,SPN,8/13/2019 0:00:00,213
6890,742,MARKET/TVA,8/13/2019 0:00:00,211
6895,742,MARKET/TVA,8/8/2019 0:00:00,210


##### Period Total

In [60]:
apc_pick = apc[apc['SURVEY_DATE'].isin(dates_pick)]
apc_pick[['STOP_ID', 'MAIN_CROSS_STREET']]
station_boards = apc_pick.groupby(['STOP_ID', 'MAIN_CROSS_STREET'])['SERIAL_NUMBER'].count().reset_index(name='BOARDS')
station_boards.sort_values(by='BOARDS', ascending=False).head(20)

,STOP_ID,MAIN_CROSS_STREET,BOARDS
35,100075,SPN,1550
1031,742,MARKET/TVA,1418
416,145,MARKET/12TH,1313
359,1351,MARKET/4TH,1221
43,100133,Mkt 4th,1082
495,163,MARKET/11TH PATTEN TOWERS,1082
470,1555,MARKET/6TH,1080
258,12,MARKET/FAMILY DOLLAR,1076
36,100077,SPS,1060
463,1539,10TH/LIBRARY,1040


##### Some checks

In [61]:
apc[apc['MAIN_CROSS_STREET']=='MARKET/4TH']

,SERIAL_NUMBER,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,...,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP,route_id,route_short_name,trip_start_astime,TOD
12,4763669,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,743,10.60,...,NaN,NaN,0,NaN,NaN,2,10C,CAMPBELL,07:46:00,AM
171,4763670,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,676,10.90,...,NaN,NaN,0,NaN,NaN,3,10G,GLENWOOD,08:55:00,AM
172,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,1,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
217,4763672,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,745,10.60,...,NaN,NaN,0,NaN,NaN,2,10C,CAMPBELL,15:56:00,PM
353,4763673,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,671,10.10,...,NaN,NaN,0,NaN,NaN,3,10A,AVON,17:05:00,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433611,4784782,109,May19 (Weekday),May 5 2019,8/9/2019 0:00:00,2,1,3,905,4.00,...,NaN,NaN,0,NaN,NaN,2,4,EASTGATE/HAMILTON PL,13:05:00,OP
439337,4785192,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,910,4.00,...,NaN,NaN,0,NaN,NaN,2,4,EASTGATE/HAMILTON PL,05:21:00,OP
439531,4785193,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,904,4.00,...,NaN,NaN,0,NaN,NaN,3,4,EASTGATE/HAMILTON PL,06:30:00,AM
439532,4785194,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,902,4.00,...,NaN,NaN,0,NaN,NaN,1,4,EASTGATE/HAMILTON PL,07:30:00,AM


In [62]:
apc[apc['MAIN_CROSS_STREET']=='Mkt 4th']

,SERIAL_NUMBER,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,...,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP,route_id,route_short_name,trip_start_astime,TOD
170,4763670,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,676,10.90,...,NaN,NaN,0,NaN,NaN,3,10G,GLENWOOD,08:55:00,AM
173,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,1,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
352,4763673,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,671,10.10,...,NaN,NaN,0,NaN,NaN,3,10A,AVON,17:05:00,PM
355,4763674,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,780,780.00,...,NaN,NaN,0,NaN,NaN,1,3,ENTERPRISE SOUTH,17:35:00,PM
397,4763675,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,782,780.00,...,NaN,NaN,0,NaN,NaN,2,3,ENTERPRISE SOUTH,18:15:00,OP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433612,4784782,109,May19 (Weekday),May 5 2019,8/9/2019 0:00:00,2,1,3,905,4.00,...,NaN,NaN,0,NaN,NaN,2,4,EASTGATE/HAMILTON PL,13:05:00,OP
439338,4785192,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,910,4.00,...,NaN,NaN,0,NaN,NaN,2,4,EASTGATE/HAMILTON PL,05:21:00,OP
439530,4785193,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,904,4.00,...,NaN,NaN,0,NaN,NaN,3,4,EASTGATE/HAMILTON PL,06:30:00,AM
439533,4785194,109,May19 (Weekday),May 5 2019,8/16/2019 0:00:00,2,1,3,902,4.00,...,NaN,NaN,0,NaN,NaN,1,4,EASTGATE/HAMILTON PL,07:30:00,AM


In [73]:
apc.iloc[172]
apc.iloc[173]

SERIAL_NUMBER                              4763671
SCHEDULE_ID                                    109
SCHEDULE_NAME                      May19 (Weekday)
SIGNUP_NAME                             May 5 2019
SURVEY_DATE                       8/1/2019 0:00:00
                                 ...              
FIRST_LAST_STOP                                  1
route_id                                         7
route_short_name     CHATTANOOGA HOUSING AUTHORITY
trip_start_astime                         09:30:00
TOD                                             OP
Name: 172, Length: 76, dtype: object

In [74]:
apc[apc['SERIAL_NUMBER']==4763671]

,SERIAL_NUMBER,SCHEDULE_ID,SCHEDULE_NAME,SIGNUP_NAME,SURVEY_DATE,SURVEY_STATUS,SURVEY_TYPE,SURVEY_SOURCE,PATTERN_ID,ROUTE_NUMBER,...,NR_BOARD,NR_ALIGHT,KNEELS,COMMENT_NUMBER,CHECKER_TIME,FIRST_LAST_STOP,route_id,route_short_name,trip_start_astime,TOD
172,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,1,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
173,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,1,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
174,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
175,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
176,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
177,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
178,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
179,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
180,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP
181,4763671,109,May19 (Weekday),May 5 2019,8/1/2019 0:00:00,2,1,3,433,7.00,...,NaN,NaN,0,NaN,NaN,2,7,CHATTANOOGA HOUSING AUTHORITY,09:30:00,OP


In [101]:
not_matches['MAIN_CROSS_STREET'].unique()

array(['Mkt41', 'StuDod-1', '58+JerPk', 'StuCha-1', 'OKb31', 'Stu+Dod',
       'DodHlth', '49thAng', 'WilMcC1', 'ShoCART', 'WilMcC0', '4thAFou1',
       '6thBroa', '56thSE', 'HixDar1', 'NPBHmDe', '12thMar1', 'Ngate',
       '10thMkt', 'HoltWil', 'Cromwell', 'HoltWilc', 'HalsCent',
       'NMktSaw', 'BoySrN1', 'CherMan', 'Mkt61', 'ChattSt', 'BailWill',
       'BraGerm', 'ShoCAR0', 'WilGre-1', 'Mkt60', 'SPN', 'SPS', 'ShoCAR1',
       'Neighbor', 'Mkt260', 'RoaWil0', '3rdCent', 'ChamCru0', 'Bro6th',
       'Mkt4th', 'ShalPO1', 'CromRRM1', 'CromCai1', '38thHig1', 'MktCC1',
       'McCWll1', 'MainLyl1', 'RivCtr', 'Riv+Wil', 'Mkt121', 'HixFal0',
       '3rdPal', 'Bra+Ger', 'NewGar', 'Mkt5th', 'CHA', 'CHA1', 'AirLee-0',
       'AirLee-1', 'DeeEmm1', 'Mk112th', 'BroChg', 'TunShall', '3rd+OK',
       'CARTAG', 'MktHou', 'StaSt', 'CMA', 'NurHo1-0', 'ShoCam0',
       'MktKin', '3rdCen0', 'BoOkAdd0', 'BonOak', 'MktCrtH', 'FOOCIT',
       'BraWal-0', 'FooEBr', 'GreWal-1', 'ShalMoo', 'Mkt6th', 'HamM

In [93]:
not_matches.sort_values(by='count', ascending=False)

,STOP_ID,MAIN_CROSS_STREET,count,stop_id,stop_name
35,100075,SPN,3686,NaN,NaN
36,100077,SPS,2689,NaN,NaN
44,100133,Mkt4th,2450,NaN,NaN
54,100189,Mkt121,1385,NaN,NaN
86,100255,HamMall,1177,NaN,NaN
...,...,...,...,...,...
82,100249,FooEBr,17,NaN,NaN
27,100058,Mkt61,16,NaN,NaN
38,100087,Neighbor,15,NaN,NaN
1,100004,Mkt41,15,NaN,NaN


In [100]:
not_matches['count'].sum()

33301

In [106]:
print('Not Matches:', not_matches['count'].sum())
print('ALL APC:', len(apc))

print(apc.groupby('route_short_name')['SERIAL_NUMBER'].count())
list(apc)

Not Matches: 33301
ALL APC: 440538
route_short_name
ALTON PARK                        42550
AMNICOLA HWY CHATT STATE           5586
AVON                              22859
CAMPBELL                           7207
CHATTANOOGA HOUSING AUTHORITY      4618
CROMWELL ROAD                     10754
DOWNTOWN SHUTTLE                  49182
EAST LAKE                         40491
EASTDALE                          16483
EASTGATE/HAMILTON PL             115103
ENTERPRISE SOUTH                   3151
GLENWOOD                          32631
GOLDEN GATEWAY                    14453
NORTH CHATT                        5258
NORTH SHORE SHUTTLE                7065
NORTHGATE OB                      38458
ROSSVILLE                         18330
ST. ELMO                           6359
Name: SERIAL_NUMBER, dtype: int64


['SERIAL_NUMBER',
 'SCHEDULE_ID',
 'SCHEDULE_NAME',
 'SIGNUP_NAME',
 'SURVEY_DATE',
 'SURVEY_STATUS',
 'SURVEY_TYPE',
 'SURVEY_SOURCE',
 'PATTERN_ID',
 'ROUTE_NUMBER',
 'ROUTE_NAME',
 'DIRECTION_NAME',
 'BRANCH',
 'SERVICE_CODE',
 'SERVICE_TYPE',
 'SERVICE_CLASS',
 'SERVICE_MODE',
 'TRIP_START_TIME',
 'TIME_PERIOD',
 'SERVICE_PERIOD',
 'TRIP_NUMBER',
 'TRIP_KEY',
 'BLOCK_NUMBER',
 'BLOCK_KEY',
 'BLOCK_ID',
 'BLOCK_NAME',
 'RUN_NUMBER',
 'RUN_KEY',
 'VEHICLE_NUMBER',
 'VEHICLE_DESCRIPTION',
 'VEHICLE_SEATS',
 'REVENUE_START',
 'REVENUE_END',
 'REVENUE_NET',
 'ODOM_START',
 'ODOM_END',
 'ODOM_NET',
 'CONDITION_NUMBER',
 'CHECKER_NAME',
 'GARAGE_NAME',
 'DIVISION_NAME',
 'OPERATOR_ID',
 'FAREBOX',
 'MATCH_COUNT',
 'COMMENTS',
 'SORT_ORDER',
 'STOP_ID',
 'MAIN_CROSS_STREET',
 'TRAVEL_DIRECTION',
 'TIMEPOINT',
 'SEGMENT_MILES',
 'TIME_SCHEDULED',
 'TIME_ACTUAL_ARRIVE',
 'TIME_ACTUAL_DEPART',
 'DWELL_TIME',
 'RUNNING_TIME_ACTUAL',
 'PASSENGERS_ON',
 'PASSENGERS_OFF',
 'PASSENGERS_IN',
 'PASS